In [1]:
import sys
sys.path.append("..")

import numpy as np
import glob
import matplotlib.pyplot as plt
import pandas as pd
#from spiralcutter_advanced.libs.spiral_funcs import *
from spiralcutter_advanced.libs.twod_funcs_mod import *
from spiralcutter_advanced.libs.profile_funcs import *
from spiralcutter_advanced.libs.helper_funcs import *
from scipy.optimize import curve_fit
from astropy.io import fits

In [20]:
def calc_chidiff(table, ml, name1, name2, cond1 = None, cond2 = None, mul1 = None, mul2 = None):
    if cond1 is None:
        cond1 = (ml == name1)
    if cond2 is None:
        cond2 = (ml == name2)
    if mul1 is not None:
        cond1 = cond1 * mul1
    if mul2 is not None:
        cond2 = cond2 * mul2
    chidmean = np.nanmean(table[cond1] / table[cond2])
    #chidstd = np.nanstd(table[cond1] / table[cond2])
    chidq25 = "{-" + str(np.round(chidmean - np.nanquantile(table[cond1] / table[cond2], 0.25), 3)) + "}"
    chidq75 = "{+" + str(np.round(np.nanquantile(table[cond1] / table[cond2], 0.75) - chidmean, 3)) + "}"
    #print(f"${np.round(chidmean, 3)}^{chidq75}_{chidq25}$")
    print(np.round(chidmean, 3))
    print()
    

gals = []
gal_paths = glob.glob("../galaxies_images/*")
for path in gal_paths:
    gals.append(path.split("/")[-1])
gals = np.sort(gals)

phi_list = []
col_list = []
model_list = []
chisqs_list = []
Ns_list = []
ctrs_list = []
#popts_list = []

bad_arms = {
        "NGC0613": [],
        "NGC0628": [],
        "NGC0986": [],
        "NGC1042": [],
        "NGC1073": [],
        "NGC1232": ["blue", "green", "yellow"],
        "NGC1300": ["blue"],
        "NGC1566": ["blue", "cyan"],
        "NGC1672": ["yellow"],
        "NGC3184": ["cyan", "orange"],
        "NGC4123": [],
        "NGC4254": ["cyan", "yellow"],
        "NGC4303": ["yellow"],
        "NGC4321": ["blue", "magenta", "yellow"],
        "NGC4535": ["orange", "teal", "yellow"],
        "NGC5085": [],
        "NGC5236": ["cyan", "magenta", "yellow"],
        "NGC5247": [],
        "NGC7412": []
}

counter = -1
col_prev = None

for gal in gals:
    path_res = f"../galaxies_images/{gal}/spiral_params/profiles_2d_final.dat"
    
    with open(path_res, "r") as file:
        for line in file:
            col, fname, chisq, N = line.split()[0:4]
            
            if col in bad_arms[gal]:
                continue
            
            N = int(N)
            if chisq == "[nan]":
                chisq = np.nan
            else:
                chisq = float(chisq)
            
            arm_str = fits.getdata(f"../galaxies_images/{gal}/str_arms_azavg/arm_str_{col}.fits")
            lpsi = len(arm_str[0, :]) / 2
            
            if not col_prev == col:
                counter += 1
            col_prev = col
            
            phi_list.append(lpsi)
            col_list.append(col)
            model_list.append(fname)
            chisqs_list.append(chisq)
            Ns_list.append(N)
            ctrs_list.append(counter)
            
phi_list = np.array(phi_list)
col_list = np.array(col_list)
model_list = np.array(model_list)
chisqs_list = np.array(chisqs_list)
Ns_list = np.array(Ns_list)
ctrs_list = np.array(ctrs_list)


#print(phi_list[model_list == "main_func"])
dips_inds = (model_list == "main_func_d1") + (model_list == "main_func_d2")
dips = (ctrs_list[dips_inds])
arms_wdips = np.zeros(len(ctrs_list)).astype(bool)
for i in range(len(dips)):
    arms_wdips += (ctrs_list == dips[i])
    
dips_inds1 = (model_list == "main_func_d1")# + (model_list == "main_func_d2")
dips1 = (ctrs_list[dips_inds1])
arms_wdips1 = np.zeros(len(ctrs_list)).astype(bool)
for i in range(len(dips1)):
    arms_wdips1 += (ctrs_list == dips1[i])
    
dips_inds2 = (model_list == "main_func_d2")# + (model_list == "main_func_d2")
dips2 = (ctrs_list[dips_inds2])
arms_wdips2 = np.zeros(len(ctrs_list)).astype(bool)
for i in range(len(dips2)):
    arms_wdips2 += (ctrs_list == dips2[i])
    
long_arms = phi_list > 180

calc_chidiff(chisqs_list, model_list, "main_func_cbr", "main_func")
calc_chidiff(chisqs_list, model_list, "main_func_sharp", "main_func")
calc_chidiff(chisqs_list, model_list, "main_func_cw", "main_func")
calc_chidiff(chisqs_list, model_list, "main_func_nozpw", "main_func")
calc_chidiff(chisqs_list, model_list, "main_func_w_asymm", "main_func")
calc_chidiff(chisqs_list, model_list, "main_func_w_n", "main_func")
calc_chidiff(chisqs_list, model_list, "main_func_add", "main_func")
calc_chidiff(chisqs_list, model_list, "", "",
             cond1 = (arms_wdips * dips_inds), cond2 = (arms_wdips * (model_list == "main_func")))
print("long:")
calc_chidiff(chisqs_list, model_list, "main_func_cbr", "main_func", mul1 = long_arms, mul2 = long_arms)
calc_chidiff(chisqs_list, model_list, "main_func_sharp", "main_func", mul1 = long_arms, mul2 = long_arms)
calc_chidiff(chisqs_list, model_list, "main_func_cw", "main_func", mul1 = long_arms, mul2 = long_arms)
calc_chidiff(chisqs_list, model_list, "main_func_nozpw", "main_func", mul1 = long_arms, mul2 = long_arms)
calc_chidiff(chisqs_list, model_list, "main_func_w_asymm", "main_func", mul1 = long_arms, mul2 = long_arms)
calc_chidiff(chisqs_list, model_list, "main_func_w_n", "main_func", mul1 = long_arms, mul2 = long_arms)
calc_chidiff(chisqs_list, model_list, "main_func_add", "main_func", mul1 = long_arms, mul2 = long_arms)
calc_chidiff(chisqs_list, model_list, "", "",
             cond1 = (arms_wdips * dips_inds), cond2 = (arms_wdips * (model_list == "main_func")),
             mul1 = long_arms, mul2 = long_arms)

1.017

1.196

1.12

1.018

0.991

0.983

0.972

0.817

long:
1.012

1.195

1.086

1.021

0.99

0.985

0.961

0.817



In [10]:
print("const_cbr vs main chi2")
print(np.nanmean(chisqs_list[model_list == "main_func_cbr"] / chisqs_list[model_list == "main_func"]))
print()
print("sharp vs main chi2")
print(np.nanmean(chisqs_list[model_list == "main_func_sharp"] / chisqs_list[model_list == "main_func"]))
print()
print("const_w vs main chi2")
print(np.nanmean(chisqs_list[model_list == "main_func_cw"] / chisqs_list[model_list == "main_func"]))
print()
print("const_nozpw vs main chi2")
print(np.nanmean(chisqs_list[model_list == "main_func_nozpw"] / chisqs_list[model_list == "main_func"]))
print()
print("w_asymm vs main chi2")
print(np.nanmean(chisqs_list[model_list == "main_func_w_asymm"] / chisqs_list[model_list == "main_func"]))
print()
print("w_n vs main chi2")
print(np.nanmean(chisqs_list[model_list == "main_func_w_n"] / chisqs_list[model_list == "main_func"]))
print()
print("add vs main chi2")
print(np.nanmean(chisqs_list[model_list == "main_func_add"] / chisqs_list[model_list == "main_func"]))
print()
print("dips chi2 vs main chi2")
print(np.nanmean(chisqs_list[arms_wdips * dips_inds] / chisqs_list[arms_wdips * (model_list == "main_func")]))
print()
print()
print()
print("add vs main chi2 (when dips present)")
print(np.nanmean(chisqs_list[arms_wdips * (model_list == "main_func_add")] / chisqs_list[arms_wdips * (model_list == "main_func")]))
print()
print("dips1 chi2 vs main chi2")
print(np.nanmean(chisqs_list[arms_wdips1 * dips_inds1] / chisqs_list[arms_wdips1 * (model_list == "main_func")]))
print()

print("========")
print("for long arms")
print("========")

print("const_cbr vs main chi2")
print(np.nanmean(chisqs_list[long_arms * (model_list == "main_func_cbr")] / chisqs_list[long_arms * (model_list == "main_func")]))
print()
print("sharp vs main chi2")
print(np.nanmean(chisqs_list[long_arms * (model_list == "main_func_sharp")] / chisqs_list[long_arms * (model_list == "main_func")]))
print()
print("const_w vs main chi2")
print(np.nanmean(chisqs_list[long_arms * (model_list == "main_func_cw")] / chisqs_list[long_arms * (model_list == "main_func")]))
print()
print("const_nozpw vs main chi2")
print(np.nanmean(chisqs_list[long_arms * (model_list == "main_func_nozpw")] / chisqs_list[long_arms * (model_list == "main_func")]))
print()
print("w_asymm vs main chi2")
print(np.nanmean(chisqs_list[long_arms * (model_list == "main_func_w_asymm")] / chisqs_list[long_arms * (model_list == "main_func")]))
print()
print("w_n vs main chi2")
print(np.nanmean(chisqs_list[long_arms * (model_list == "main_func_w_n")] / chisqs_list[long_arms * (model_list == "main_func")]))
print()
print("add vs main chi2")
print(np.nanmean(chisqs_list[long_arms * (model_list == "main_func_add")] / chisqs_list[long_arms * (model_list == "main_func")]))
print()
print("dips chi2 vs main chi2")
print(np.nanmean(chisqs_list[long_arms * arms_wdips * dips_inds] / chisqs_list[long_arms * arms_wdips * (model_list == "main_func")]))
print()
print()
print("add vs main chi2 (when dips present)")
print(np.nanmean(chisqs_list[long_arms * arms_wdips * (model_list == "main_func_add")] / chisqs_list[long_arms * arms_wdips * (model_list == "main_func")]))
print()

funcs = [main_func_cw, main_func_nozpw, main_func_cbr, main_func_sharp, main_func, main_func_w_asymm, main_func_w_n,
         main_func_add, main_func_d1, main_func_d2]
fns = []
for i in range(len(funcs)):
    func = funcs[i]
    fns.append(func.__name__)
pn = [9, 9, 9, 8, 10, 12, 13, 12, 13, 16]
fns = np.array(fns)

bics = []
for i in range(len(chisqs_list)):
    chisq = chisqs_list[i]
    model = model_list[i]
    N = Ns_list[i]
    pnum = pn[np.where(fns == model)[0][0]]
    
    bic = chisq * N + (pnum * np.log(N))
    bics.append(bic)

bics = np.array(bics)
    
print("(6) w_n vs main chi2")
print(chisqs_list[model_list == "main_func_w_n"] / chisqs_list[model_list == "main_func"])
print(bics[model_list == "main_func_w_n"] / bics[model_list == "main_func"])
print()

const_cbr vs main chi2
1.0172806506774057

sharp vs main chi2
1.1960867213794406

const_w vs main chi2
1.1197853011311605

const_nozpw vs main chi2
1.018473406400095

w_asymm vs main chi2
0.9906025708265226

w_n vs main chi2
0.9834856252407059

add vs main chi2
0.9717538392114027

dips chi2 vs main chi2
0.8166017675622425



add vs main chi2 (when dips present)
0.9541445289318893

dips1 chi2 vs main chi2
0.8377460693678198

for long arms
const_cbr vs main chi2
1.011945065827592

sharp vs main chi2
1.1952100971045874

const_w vs main chi2
1.0859175489911193

const_nozpw vs main chi2
1.0210238752679746

w_asymm vs main chi2
0.9901270633365967

w_n vs main chi2
0.9845590756881174

add vs main chi2
0.9610510033222144

dips chi2 vs main chi2
0.8166017675622425


add vs main chi2 (when dips present)
0.9541445289318893

(6) w_n vs main chi2
[0.99496883 0.997779   0.98581121 0.99584304 0.90517856        nan
 0.99544953 0.99718509 0.93060644 0.98363379 0.99842328 0.99985912
 0.98641916 0.988693

# masked

In [21]:
gals = []
gal_paths = glob.glob("../galaxies_images/*")
for path in gal_paths:
    gals.append(path.split("/")[-1])
gals = np.sort(gals)

phi_list = []
col_list = []
model_list = []
chisqs_list = []
Ns_list = []
ctrs_list = []
#popts_list = []

bad_arms = {
        "NGC0613": [],
        "NGC0628": [],
        "NGC0986": [],
        "NGC1042": [],
        "NGC1073": [],
        "NGC1232": ["blue", "green", "yellow"],
        "NGC1300": ["blue"],
        "NGC1566": ["blue", "cyan"],
        "NGC1672": ["yellow"],
        "NGC3184": ["cyan", "orange"],
        "NGC4123": [],
        "NGC4254": ["cyan", "yellow"],
        "NGC4303": ["yellow"],
        "NGC4321": ["blue", "magenta", "yellow"],
        "NGC4535": ["orange", "teal", "yellow"],
        "NGC5085": [],
        "NGC5236": ["cyan", "magenta", "yellow"],
        "NGC5247": [],
        "NGC7412": []
}

counter = -1
col_prev = None

for gal in gals:
    path_res = f"../galaxies_images/{gal}/spiral_params/profiles_2d_final_masked.dat"
    
    with open(path_res, "r") as file:
        for line in file:
            col, fname, chisq, N = line.split()[0:4]
            
            if col in bad_arms[gal]:
                continue
            
            N = int(N)
            if chisq == "[nan]":
                chisq = np.nan
            else:
                chisq = float(chisq)
            
            arm_str = fits.getdata(f"../galaxies_images/{gal}/str_arms_azavg_masked/arm_str_{col}.fits")
            lpsi = len(arm_str[0, :]) / 2
            
            if not col_prev == col:
                counter += 1
            col_prev = col
            
            phi_list.append(lpsi)
            col_list.append(col)
            model_list.append(fname)
            chisqs_list.append(chisq)
            Ns_list.append(N)
            ctrs_list.append(counter)
            
phi_list = np.array(phi_list)
col_list = np.array(col_list)
model_list = np.array(model_list)
chisqs_list = np.array(chisqs_list)
Ns_list = np.array(Ns_list)
ctrs_list = np.array(ctrs_list)


#print(phi_list[model_list == "main_func"])
dips_inds = (model_list == "main_func_d1") + (model_list == "main_func_d2")
dips = (ctrs_list[dips_inds])
arms_wdips = np.zeros(len(ctrs_list)).astype(bool)
for i in range(len(dips)):
    arms_wdips += (ctrs_list == dips[i])

dips_inds1 = (model_list == "main_func_d1")# + (model_list == "main_func_d2")
dips1 = (ctrs_list[dips_inds1])
arms_wdips1 = np.zeros(len(ctrs_list)).astype(bool)
for i in range(len(dips1)):
    arms_wdips1 += (ctrs_list == dips1[i])
    
dips_inds2 = (model_list == "main_func_d2")# + (model_list == "main_func_d2")
dips2 = (ctrs_list[dips_inds2])
arms_wdips2 = np.zeros(len(ctrs_list)).astype(bool)
for i in range(len(dips2)):
    arms_wdips2 += (ctrs_list == dips2[i])
    
long_arms = phi_list > 180

calc_chidiff(chisqs_list, model_list, "main_func_cbr", "main_func")
calc_chidiff(chisqs_list, model_list, "main_func_sharp", "main_func")
calc_chidiff(chisqs_list, model_list, "main_func_cw", "main_func")
calc_chidiff(chisqs_list, model_list, "main_func_nozpw", "main_func")
calc_chidiff(chisqs_list, model_list, "main_func_w_asymm", "main_func")
calc_chidiff(chisqs_list, model_list, "main_func_w_n", "main_func")
calc_chidiff(chisqs_list, model_list, "main_func_add", "main_func")
calc_chidiff(chisqs_list, model_list, "", "",
             cond1 = (arms_wdips * dips_inds), cond2 = (arms_wdips * (model_list == "main_func")))
print("long:")
calc_chidiff(chisqs_list, model_list, "main_func_cbr", "main_func", mul1 = long_arms, mul2 = long_arms)
calc_chidiff(chisqs_list, model_list, "main_func_sharp", "main_func", mul1 = long_arms, mul2 = long_arms)
calc_chidiff(chisqs_list, model_list, "main_func_cw", "main_func", mul1 = long_arms, mul2 = long_arms)
calc_chidiff(chisqs_list, model_list, "main_func_nozpw", "main_func", mul1 = long_arms, mul2 = long_arms)
calc_chidiff(chisqs_list, model_list, "main_func_w_asymm", "main_func", mul1 = long_arms, mul2 = long_arms)
calc_chidiff(chisqs_list, model_list, "main_func_w_n", "main_func", mul1 = long_arms, mul2 = long_arms)
calc_chidiff(chisqs_list, model_list, "main_func_add", "main_func", mul1 = long_arms, mul2 = long_arms)
calc_chidiff(chisqs_list, model_list, "", "",
             cond1 = (arms_wdips * dips_inds), cond2 = (arms_wdips * (model_list == "main_func")),
             mul1 = long_arms, mul2 = long_arms)

1.021

1.137

1.043

1.008

0.987

0.982

0.96

0.829

long:
1.034

1.118

1.024

1.01

0.987

0.979

0.959

0.829



In [ ]:
print("(8) dips chi2 vs main chi2")
print(np.nanmean(chisqs_list[arms_wdips * dips_inds] / chisqs_list[arms_wdips * (model_list == "main_func")]))
print()
print("(8') dips2 chi2 vs main chi2")
print(np.nanmean(chisqs_list[arms_wdips2 * dips_inds2] / chisqs_list[arms_wdips2 * (model_list == "main_func")]))
print()
print("(7) add vs main chi2")
print(np.nanmean(chisqs_list[model_list == "main_func_add"] / chisqs_list[model_list == "main_func"]))
print()
print("add vs main chi2 (when dips present)")
print(np.nanmean(chisqs_list[arms_wdips * (model_list == "main_func_add")] / chisqs_list[arms_wdips * (model_list == "main_func")]))
print()
print("(5) w_asymm vs main chi2")
print(np.nanmean(chisqs_list[model_list == "main_func_w_asymm"] / chisqs_list[model_list == "main_func"]))
print()
print("(6) w_n vs main chi2")
print(np.nanmean(chisqs_list[model_list == "main_func_w_n"] / chisqs_list[model_list == "main_func"]))
print()
print("(2) sharp vs main chi2")
print(np.nanmean(chisqs_list[model_list == "main_func_sharp"] / chisqs_list[model_list == "main_func"]))
print()
print("(3) const_w vs main chi2")
print(np.nanmean(chisqs_list[model_list == "main_func_cw"] / chisqs_list[model_list == "main_func"]))
print()
print("(4) const_nozpw vs main chi2")
print(np.nanmean(chisqs_list[model_list == "main_func_nozpw"] / chisqs_list[model_list == "main_func"]))
print()
print("(1) const_cbr vs main chi2")
print(np.nanmean(chisqs_list[model_list == "main_func_cbr"] / chisqs_list[model_list == "main_func"]))
print()

print("========")
print("for long arms")
print("========")

print("dips chi2 vs main chi2")
print(np.nanmean(chisqs_list[long_arms * arms_wdips * dips_inds] / chisqs_list[long_arms * arms_wdips * (model_list == "main_func")]))
print()
print("add vs main chi2")
print(np.nanmean(chisqs_list[long_arms * (model_list == "main_func_add")] / chisqs_list[long_arms * (model_list == "main_func")]))
print()
print("add vs main chi2 (when dips present)")
print(np.nanmean(chisqs_list[long_arms * arms_wdips * (model_list == "main_func_add")] / chisqs_list[long_arms * arms_wdips * (model_list == "main_func")]))
print()
print("w_asymm vs main chi2")
print(np.nanmean(chisqs_list[long_arms * (model_list == "main_func_w_asymm")] / chisqs_list[long_arms * (model_list == "main_func")]))
print()
print("w_n vs main chi2")
print(np.nanmean(chisqs_list[long_arms * (model_list == "main_func_w_n")] / chisqs_list[long_arms * (model_list == "main_func")]))
print()
print("sharp vs main chi2")
print(np.nanmean(chisqs_list[long_arms * (model_list == "main_func_sharp")] / chisqs_list[long_arms * (model_list == "main_func")]))
print()
print("const_w vs main chi2")
print(np.nanmean(chisqs_list[long_arms * (model_list == "main_func_cw")] / chisqs_list[long_arms * (model_list == "main_func")]))
print()
print("const_nozpw vs main chi2")
print(np.nanmean(chisqs_list[long_arms * (model_list == "main_func_nozpw")] / chisqs_list[long_arms * (model_list == "main_func")]))
print()
print("const_cbr vs main chi2")
print(np.nanmean(chisqs_list[long_arms * (model_list == "main_func_cbr")] / chisqs_list[long_arms * (model_list == "main_func")]))
print()

funcs = [main_func_cw, main_func_nozpw, main_func_cbr, main_func_sharp, main_func, main_func_w_asymm, main_func_w_n,
         main_func_add, main_func_d1, main_func_d2]
fns = []
for i in range(len(funcs)):
    func = funcs[i]
    fns.append(func.__name__)
pn = [9, 9, 9, 8, 10, 12, 13, 12, 13, 16]
fns = np.array(fns)

bics = []
for i in range(len(chisqs_list)):
    chisq = chisqs_list[i]
    model = model_list[i]
    N = Ns_list[i]
    pnum = pn[np.where(fns == model)[0][0]]
    
    bic = chisq * N + (pnum * np.log(N))
    bics.append(bic)

bics = np.array(bics)
    
print("(6) w_n vs main chi2")
print(chisqs_list[model_list == "main_func_w_n"] / chisqs_list[model_list == "main_func"])
print(bics[model_list == "main_func_w_n"] / bics[model_list == "main_func"])
print()

# uv

In [4]:
gals = []
gal_paths = glob.glob("../gals_uv/*")
for path in gal_paths:
    gals.append(path.split("/")[-1])
gals = np.sort(gals)

phi_list = []
col_list = []
model_list = []
chisqs_list = []
Ns_list = []
ctrs_list = []
#popts_list = []

bad_arms = {
        "NGC0613": ["red"],
        "NGC0628": ["blue", "red"],
        "NGC0986": [],
        "NGC1042": [],
        "NGC1073": ["cyan"],
        "NGC1232": ["cyan"],
        "NGC1300": ["blue", "red"],
        "NGC1566": ["blue", "cyan", "green", "red"],
        "NGC1672": ["blue", "cyan", "red", "yellow"],
        "NGC3184": ["blue", "cyan", "orange"],
        "NGC4123": ["blue", "red"],
        "NGC4254": ["cyan"],
        "NGC4303": ["blue", "cyan", "red", "yellow"],
        "NGC4321": ["blue", "cyan", "magenta", "teal", "yellow"],
        "NGC4535": ["teal"],
        "NGC5085": ["blue"],
        "NGC5236": ["cyan", "magenta", "orange", "yellow"],
        "NGC5247": ["cyan"],
        "NGC7412": ["blue", "cyan", "magenta"]
}

counter = -1
col_prev = None

for gal in gals:
    path_res = f"../gals_uv/{gal}/spiral_params/profiles_2d_final.dat"
    
    with open(path_res, "r") as file:
        for line in file:
            col, fname, chisq, N = line.split()[0:4]
            
            if col in bad_arms[gal]:
                continue
            
            N = int(N)
            if chisq == "[nan]":
                chisq = np.nan
            else:
                chisq = float(chisq)
            
            arm_str = fits.getdata(f"../gals_uv/{gal}/str_arms_azavg/arm_str_{col}.fits")
            lpsi = len(arm_str[0, :]) / 2
            
            if not col_prev == col:
                counter += 1
            col_prev = col
            
            phi_list.append(lpsi)
            col_list.append(col)
            model_list.append(fname)
            chisqs_list.append(chisq)
            Ns_list.append(N)
            ctrs_list.append(counter)
            
phi_list = np.array(phi_list)
col_list = np.array(col_list)
model_list = np.array(model_list)
chisqs_list = np.array(chisqs_list)
Ns_list = np.array(Ns_list)
ctrs_list = np.array(ctrs_list)


#print(phi_list[model_list == "main_func"])
dips_inds = (model_list == "main_func_d1") + (model_list == "main_func_d2")
dips = (ctrs_list[dips_inds])
arms_wdips = np.zeros(len(ctrs_list)).astype(bool)
for i in range(len(dips)):
    arms_wdips += (ctrs_list == dips[i])
    
dips_inds1 = (model_list == "main_func_d1")# + (model_list == "main_func_d2")
dips1 = (ctrs_list[dips_inds1])
arms_wdips1 = np.zeros(len(ctrs_list)).astype(bool)
for i in range(len(dips1)):
    arms_wdips1 += (ctrs_list == dips1[i])
    
dips_inds2 = (model_list == "main_func_d2")# + (model_list == "main_func_d2")
dips2 = (ctrs_list[dips_inds2])
arms_wdips2 = np.zeros(len(ctrs_list)).astype(bool)
for i in range(len(dips2)):
    arms_wdips2 += (ctrs_list == dips2[i])
    
long_arms = phi_list > 180

print("const_cbr vs main chi2")
print(np.nanmean(chisqs_list[model_list == "main_func_cbr"] / chisqs_list[model_list == "main_func"]))
print()
print("sharp vs main chi2")
print(np.nanmean(chisqs_list[model_list == "main_func_sharp"] / chisqs_list[model_list == "main_func"]))
print()
print("const_w vs main chi2")
print(np.nanmean(chisqs_list[model_list == "main_func_cw"] / chisqs_list[model_list == "main_func"]))
print()
print("const_nozpw vs main chi2")
print(np.nanmean(chisqs_list[model_list == "main_func_nozpw"] / chisqs_list[model_list == "main_func"]))
print()
print("w_asymm vs main chi2")
print(np.nanmean(chisqs_list[model_list == "main_func_w_asymm"] / chisqs_list[model_list == "main_func"]))
print()
print("w_n vs main chi2")
print(np.nanmean(chisqs_list[model_list == "main_func_w_n"] / chisqs_list[model_list == "main_func"]))
print()
print("add vs main chi2")
print(np.nanmean(chisqs_list[model_list == "main_func_add"] / chisqs_list[model_list == "main_func"]))
print()
#print("dips chi2 vs main chi2")
#print(np.nanmean(chisqs_list[arms_wdips * dips_inds] / chisqs_list[arms_wdips * (model_list == "main_func")]))
#print()
print()
print()
print("add vs main chi2 (when dips present)")
print(np.nanmean(chisqs_list[arms_wdips * (model_list == "main_func_add")] / chisqs_list[arms_wdips * (model_list == "main_func")]))
print()
print("dips1 chi2 vs main chi2")
print(np.nanmean(chisqs_list[arms_wdips1 * dips_inds1] / chisqs_list[arms_wdips1 * (model_list == "main_func")]))
print()

print("========")
print("for long arms")
print("========")

print("const_cbr vs main chi2")
print(np.nanmean(chisqs_list[long_arms * (model_list == "main_func_cbr")] / chisqs_list[long_arms * (model_list == "main_func")]))
print()
print("sharp vs main chi2")
print(np.nanmean(chisqs_list[long_arms * (model_list == "main_func_sharp")] / chisqs_list[long_arms * (model_list == "main_func")]))
print()
print("const_w vs main chi2")
print(np.nanmean(chisqs_list[long_arms * (model_list == "main_func_cw")] / chisqs_list[long_arms * (model_list == "main_func")]))
print()
print("const_nozpw vs main chi2")
print(np.nanmean(chisqs_list[long_arms * (model_list == "main_func_nozpw")] / chisqs_list[long_arms * (model_list == "main_func")]))
print()
print("w_asymm vs main chi2")
print(np.nanmean(chisqs_list[long_arms * (model_list == "main_func_w_asymm")] / chisqs_list[long_arms * (model_list == "main_func")]))
print()
print("w_n vs main chi2")
print(np.nanmean(chisqs_list[long_arms * (model_list == "main_func_w_n")] / chisqs_list[long_arms * (model_list == "main_func")]))
print()
print("add vs main chi2")
print(np.nanmean(chisqs_list[long_arms * (model_list == "main_func_add")] / chisqs_list[long_arms * (model_list == "main_func")]))
print()
#print("dips chi2 vs main chi2")
#print(np.nanmean(chisqs_list[long_arms * arms_wdips * dips_inds] / chisqs_list[long_arms * arms_wdips * (model_list == "main_func")]))
#print()
print()
print("add vs main chi2 (when dips present)")
print(np.nanmean(chisqs_list[long_arms * arms_wdips * (model_list == "main_func_add")] / chisqs_list[long_arms * arms_wdips * (model_list == "main_func")]))
print()

funcs = [main_func_cw, main_func_nozpw, main_func_cbr, main_func_sharp, main_func, main_func_w_asymm, main_func_w_n,
         main_func_add, main_func_d1, main_func_d2]
fns = []
for i in range(len(funcs)):
    func = funcs[i]
    fns.append(func.__name__)
pn = [9, 9, 9, 8, 10, 12, 13, 12, 13, 16]
fns = np.array(fns)

bics = []
for i in range(len(chisqs_list)):
    chisq = chisqs_list[i]
    model = model_list[i]
    N = Ns_list[i]
    pnum = pn[np.where(fns == model)[0][0]]
    
    bic = chisq * N + (pnum * np.log(N))
    bics.append(bic)

bics = np.array(bics)
    
print("(6) w_n vs main chi2")
print(chisqs_list[model_list == "main_func_w_n"] / chisqs_list[model_list == "main_func"])
print(bics[model_list == "main_func_w_n"] / bics[model_list == "main_func"])
print()

const_cbr vs main chi2
1.0432447410034065

sharp vs main chi2
1.1013799128040027

const_w vs main chi2
1.0137548432582453

const_nozpw vs main chi2
1.0132938304356696

w_asymm vs main chi2
0.9929804823888765

w_n vs main chi2
1.0153467442511934

add vs main chi2
0.9916957600401408



add vs main chi2 (when dips present)
1.0083575640894467

dips1 chi2 vs main chi2
0.9090176649424058

for long arms
const_cbr vs main chi2
1.0295032376076447

sharp vs main chi2
1.088998834582597

const_w vs main chi2
1.0139360882109811

const_nozpw vs main chi2
1.012784397352487

w_asymm vs main chi2
0.9902314950935205

w_n vs main chi2
1.0077924726022156

add vs main chi2
0.9951091529721822


add vs main chi2 (when dips present)
1.0083575640894467

(6) w_n vs main chi2
[0.99594871 1.04343204 0.98473509 0.98200281 0.9975579  0.95549156
 0.96751381 0.98393372 0.97001973 1.0627148  1.00210369 1.06555361
 0.99441847 0.9940639  0.99079252 0.99813313 1.10749848 0.96843087
 0.98078106 0.99605081 0.99304085 0.990